In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers
# !pip install simpletransformers

In [4]:
# import os 
# os.chdir("/content/drive/MyDrive/GP-2022") #place that have data and reader script

In [5]:
from simpletransformers.question_answering import QuestionAnsweringModel
from transformers import AutoConfig , BertForQuestionAnswering , BertTokenizer
import torch

In [10]:
config = AutoConfig.from_pretrained("F:/Gp/Question Answering/Model/outputs/bert")
model = BertForQuestionAnswering.from_pretrained("F:/Gp/Question Answering/Model/outputs/bert", config=config)

In [9]:
tokenizer = BertTokenizer.from_pretrained("F:/Gp/Question Answering/Model/outputs/bert")

In [11]:
from transformers import pipeline
qa_pipe =pipeline("question-answering",model=model,tokenizer=tokenizer)

In [3]:
import read_write_qrcd as qrcd
val=qrcd.load_jsonl("F:/Gp/Question Answering/Quran Task/Quran shared task data/qrcd_v1.1_train.jsonl")

Loaded 710 records from F:/Gp/Question Answering/Quran Task/Quran shared task data/qrcd_v1.1_train.jsonl


In [4]:
val

[{'pq_id': '2:8-16_364',
  'passage': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين. يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون. في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون. وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون. ألا إنهم هم المفسدون ولكن لا يشعرون. وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون. وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون. الله يستهزئ بهم ويمدهم في طغيانهم يعمهون. أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين.',
  'surah': 2,
  'verses': '8-16',
  'question': 'لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟',
  'answers': [{'text': 'أولئك الذين اشتروا الضلالة بالهدى',
    'start_char': 504}]},
 {'pq_id': '2:174-176_364',
  'passage': 'إن الذين يكتمون ما أنزل الله من الكتاب ويشترون به ثمنا قليلا أولئك ما يأكل

In [13]:
# qa_pipe(question=val[105]['question'] ,context=val[105]['passage'],top_k=2)
qa_pipe(question= 'ما اهو اجر الذين امنوا و عملوا الصالحات؟',context="وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر إنا أعتدنا للظالمين نارا أحاط بهم سرادقها وإن يستغيثوا يغاثوا بماء كالمهل يشوي الوجوه بئس الشراب وساءت مرتفقا. إن الذين آمنوا وعملوا الصالحات إنا لا نضيع أجر من أحسن عملا. أولئك لهم جنات عدن تجري من تحتهم الأنهار يحلون فيها من أساور من ذهب ويلبسون ثيابا خضرا من سندس وإستبرق متكئين فيها على الأرائك نعم الثواب وحسنت مرتفقا.",top_k=2)

[{'score': 0.7065602540969849,
  'start': 17,
  'end': 46,
  'answer': 'فمن شاء فليؤمن ومن شاء فليكفر'},
 {'score': 0.00807335413992405,
  'start': 32,
  'end': 46,
  'answer': 'ومن شاء فليكفر'}]

In [5]:
def prepare_data_arabert(data):
  data_prepared=[]
  answers=[] #list for list of answers for every record
  for x in data: #for record in data set
    for i in range(len(x['answers'])):#loop for records that have more than one answer
            
      answers.append({
                             'text': x['answers'][i]['text'],
                          'answer_start': x['answers'][i]['start_char']
                          })
                     
    
    dict={'context':x['passage'],
        "qas": [
            {
                'id': x['pq_id'],
            "is_impossible": False,
            'question': x['question'],
            'answers': answers #list of answers we have done in the second for loop
            },
         ],}
    if(answers==[]):
      dict["qas"][0]['is_impossible']=True
    data_prepared.append(dict)#adding record to the new list of dictioneries (data)
    answers=[] #empty list of answers for next record answers
  
  return data_prepared

In [6]:
val1=prepare_data_arabert(val)
val1

[{'context': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين. يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون. في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون. وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون. ألا إنهم هم المفسدون ولكن لا يشعرون. وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون. وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون. الله يستهزئ بهم ويمدهم في طغيانهم يعمهون. أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين.',
  'qas': [{'id': '2:8-16_364',
    'is_impossible': False,
    'question': 'لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟',
    'answers': [{'text': 'أولئك الذين اشتروا الضلالة بالهدى',
      'answer_start': 504}]}]},
 {'context': 'إن الذين يكتمون ما أنزل الله من الكتاب ويشترون به ثمنا قليلا أولئك ما يأكلون في بطونهم إلا ا